In [14]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision.models import mobilenet_v2# 加载训练集和测试集

# 转换器
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

In [17]:
# Assuming you have defined trainloader, testloader, device, and add_label_noise function
device = "mps"
# Define MobileNet model
net = mobilenet_v2(pretrained=False)
net.classifier[1] = nn.Linear(net.classifier[1].in_features, 10)
net.to(device)
print("start")

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)

# Training loop
for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Print progress
        if i % 200 == 199:  # print every 200 mini-batches
            print(f'Epoch: {epoch + 1}, Batch: {i + 1}/{len(trainloader)}, Loss: {running_loss / 200}')
            running_loss = 0.0

    print(f'Epoch {epoch + 1} completed, Loss: {running_loss / len(trainloader)}')

print('Finished Training')

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')

start
Epoch: 1, Batch: 200/782, Loss: 2.1695523554086686
Epoch: 1, Batch: 400/782, Loss: 1.9130781215429307
Epoch: 1, Batch: 600/782, Loss: 1.7893018054962158
Epoch 1 completed, Loss: 0.3958706585952388
Epoch: 2, Batch: 200/782, Loss: 1.6553655380010606
Epoch: 2, Batch: 400/782, Loss: 1.6052641868591309
Epoch: 2, Batch: 600/782, Loss: 1.5757802295684815
Epoch 2 completed, Loss: 0.35932571000760166
Epoch: 3, Batch: 200/782, Loss: 1.4877342194318772
Epoch: 3, Batch: 400/782, Loss: 1.4544546788930892
Epoch: 3, Batch: 600/782, Loss: 1.4561536186933517
Epoch 3 completed, Loss: 0.33057859715293436
Epoch: 4, Batch: 200/782, Loss: 1.3596572536230087
Epoch: 4, Batch: 400/782, Loss: 1.3739164030551911
Epoch: 4, Batch: 600/782, Loss: 1.3641989117860793
Epoch 4 completed, Loss: 0.3128522081143411
Epoch: 5, Batch: 200/782, Loss: 1.2704178071022034
Epoch: 5, Batch: 400/782, Loss: 1.2857641220092773
Epoch: 5, Batch: 600/782, Loss: 1.2795028352737428
Epoch 5 completed, Loss: 0.2995008638752696
Epoch: 

In [11]:
print(total)

10000


In [12]:
print(correct)

1865
